In [1]:
%matplotlib inline
import os
import torch
import numpy as np
import matplotlib.pyplot as plt 

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch.nn.functional as F

def extract(dp, gt, keep_dims=True):

    while dp.size() != gt.size():
        dp = F.interpolate(dp, scale_factor=(2, 2, 2), mode='trilinear')

    print(dp.size())
    vpos_sets, vneg_sets = [],[]
    for i in range(dp.size(0)):
        pos_voxels = torch.argwhere(gt[i, 0] > 0)
        neg_voxels = torch.argwhere(gt[i, 0] == 0)
        
        pos_set = dp[i, :, pos_voxels[:, 0], pos_voxels[:, 1], pos_voxels[:, 2]]
        neg_set = dp[i, :, neg_voxels[:, 0], neg_voxels[:, 1], neg_voxels[:, 2]]

        vpos_sets.append(pos_set)
        vneg_sets.append(neg_set)

    print([v.size() for v in vpos_sets])
    vpos_sets = torch.concat(vpos_sets, -1)    
    vneg_sets = torch.concat(vneg_sets, -1)

    if keep_dims: 
        return vpos_sets, vneg_sets
    
    return vpos_sets.mean(dim=0), vneg_sets.mean(dim=0)

In [3]:
from sklearn import mixture 

In [4]:
def fit_gmm(pos, neg, n_components=2):
    if isinstance(n_components, str):
        # infer the number of components
        pass 

    if not isinstance(pos, np.ndarray):
        pos = pos.detach().cpu().numpy()

    if not isinstance(neg, np.ndarray):
        neg = neg.detach().cpu().numpy()

    if len(pos.shape) < 2:
        pos = np.expand_dims(pos, 1)
        neg = np.expand_dims(neg, 1)

    clf = mixture.GaussianMixture(n_components=n_components, covariance_type="full")
    clf.fit(np.concatenate((pos, neg)))

    return clf, clf.means_, clf.covariances_

In [5]:
from sklearn import metrics

def evaluate_gmm(clf: mixture.GaussianMixture, preds, labels_true):
    # if len(preds.shape) > 1 :
    #     preds = np.squeeze(preds, 1)

    # print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, preds))
    # print("Completeness: %0.3f" % metrics.completeness_score(labels_true, preds))
    # print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, preds))
    
    sample_scores = clf.score_samples(preds)
    print(sample_scores)

In [6]:
from scipy.interpolate import UnivariateSpline

def visualize_scalar_dist(s, nbins=30, c='b', ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    p, x = np.histogram(s, bins=nbins) # bin it into n = N//10 bins
    x = x[:-1] + (x[1] - x[0])/2   # convert bin edges to centers
    f = UnivariateSpline(x, p, s=nbins)
    ax.plot(x, f(x), c=c)

    return fig, ax

def visualize_multivar_dist():
    pass

def visualize_dist(s, *args, **kwargs):
    if len(s.shape) > 1 or (len(s.shape) == 2 and s.shape[1] != 1):
        visualize_scalar_dist(s, *args, **kwargs)
    else:
        visualize_multivar_dist(s, *args, **kwargs)

In [7]:
import torch
import torch.nn.functional as F

In [8]:
class LinearClassifier(torch.nn.Module):
    def __init__(self, in_channels, num_classes, w, h, d):
        super(LinearClassifier, self).__init__()

        self.in_channels = in_channels
        self.num_classes = num_classes 
        self.width = w
        self.height = h
        self.depth = d
        self.classifier = torch.nn.Conv3d(in_channels, num_classes, (1,1,1))

    def forward(self, embeddings):
        embeddings = embeddings.reshape(-1, self.height, self.width, self.depth, self.in_channels)
        embeddings = embeddings.permute(0,4,1,2, 3)
        logits = self.classifier(embeddings)
        

        return logits


def train(model, train_dataloader, epochs, optimizer, criterion):
  # put model in training mode
  model.train()

  for epoch in range(epochs):
    print("Epoch:", epoch)
    for idx, batch in enumerate(train_dataloader):
        # pixel_values = batch["pixel_values"].to(device)
        # labels = batch["labels"].to(device)
        dp, gt = batch

        # forward pass
        out_logits = model(dp)
        if out_logits.size() != gt.size():
            while out_logits.size() != gt.size():
                out_logits = F.interpolate(out_logits, scale_factor=(2, 2, 2), mode='trilinear')
            
        dice_loss = criterion(out_logits, gt)
        dice_loss.backward()
        optimizer.step()

        # zero the parameter gradients
        optimizer.zero_grad()

def test(model, test_dataloader, criterion):
    total_loss = 0 
    for batch_idx, batch in enumerate(test_dataloader):

        dp, gt = batch 

        out_logits = model(dp)
        if out_logits.size() != gt.size():
            while out_logits.size() != gt.size():
                out_logits = F.interpolate(out_logits, scale_factor=(2, 2, 2), mode='trilinear')
            
        dice_loss = criterion(out_logits, gt)
        


In [9]:
# from torch.optim import AdamW
# from Upstream.nnunet.training.loss_functions.dice_loss import SoftDiceLoss

# model = LinearClassifier()

# train_dataloader = None


# # training hyperparameters
# # NOTE: I've just put some random ones here, not optimized at all
# # feel free to experiment, see also DINOv2 paper
# learning_rate = 5e-5
# epochs = 10

# optimizer = AdamW(model.parameters(), lr=learning_rate)
# criterion = SoftDiceLoss()

# # put model on GPU (set runtime to GPU in Google Colab)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# train(model, train_dataloader, epochs=epochs, optimizer=optimizer)

In [10]:
# Linear Probing
# train_features == dynamic prompts
from sklearn.linear_model import LogisticRegression, LinearRegression

def cast_and_format(*args):
    fmt = []
    for arg in args:
        if not isinstance(arg, np.ndarray):
            arg = arg.detach().cpu().numpy()

        bsize = arg.shape[0]

        if len(arg.shape) > 2:
            arg = np.reshape(arg, (bsize, -1))

        fmt.append(arg)

    return tuple(fmt)

def linear_probing(train_features, train_labels):
    
    train_features, train_labels = cast_and_format(train_features, train_labels)

    # classifier = LogisticRegression(random_state=0, C=0.3, max_iter=1000, verbose=1)
    classifier = LinearRegression()
    classifier.fit(train_features, train_labels)

    return classifier

def test_linear_classifier(classifier, features, labels):

    features, labels = cast_and_format(features, labels)

    predictions = classifier.predict(features)

    accuracy = np.mean((labels == predictions).astype(float)) * 100

    return predictions, accuracy


In [11]:
from nnunet.training.network_training.UniSeg_Trainer import UniSeg_Trainer
from nnunet.paths import network_training_output_dir
from nnunet.run.default_configuration import get_default_configuration

def collect_model_info_and_evaluate(model_checkpoint, exp_name = "UniSeg_Trainer", network = "3d_fullres", task = "Task097_11task", network_trainer = "UniSeg_Trainer", plans_identifier = "DoDNetPlans", fold=0):
    # Get the main plans file
    plans_file, output_folder_name, dataset_directory, batch_dice, stage, \
        trainer_class = get_default_configuration(exp_name, network, task, network_trainer, plans_identifier)
    trainer = trainer_class(plans_file, fold, output_folder=output_folder_name, dataset_directory=dataset_directory,
                            batch_dice=batch_dice, stage=stage, unpack_data=True, deterministic=True, fp16=True)
    
    # Extract the model name from the checkpoint file name
    path_checkpoint = os.path.join(output_folder_name, f"fold_{fold}",model_checkpoint)        
    trainer.load_checkpoint(path_checkpoint)
    
    # Get the data loaders
    tr_gen = trainer.tr_gen
    val_gen = trainer.val_gen
    len_data = len(tr_gen.generator._data)
    outputs, inter_mediate_prompts, dynamic_prompts, task_prompts, features_xs,targets = [], [], [], [], [], []
    for i in range(len_data):
        output, inter_mediate_prompt, dynamic_prompt, task_prompt, features_x, target = trainer.run_iteration(tr_gen, False,False, True)
        # output, inter_mediate_prompt, dynamic_prompt, task_prompt, features_x, target = trainer.run_iteration(tr_gen, False,False)
        # metric = get_metric(output, target, inter_mediate_prompt, dynamic_prompt, task_prompt, features_x)
        # Possibly move to CPU
        outputs.append(output)
        inter_mediate_prompts.append(inter_mediate_prompt)
        dynamic_prompts.append(dynamic_prompt)
        task_prompts.append(task_prompt)
        features_xs.append(features_x)
        # target is a list
        targets.append(target)

        if i == 3:
            break

    return tr_gen, trainer, outputs, inter_mediate_prompts, dynamic_prompts, task_prompts, features_xs,targets



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



In [12]:
model_checkpoint = "/data/nnUNet_trained_models/UniSeg_Trainer/3d_fullres/Task091_MOTS/UniSeg_Trainer__DoDNetPlans/fold_0/model_latest.model"
tr_gen, trainer, outputs, inter_mediate_prompts, dynamic_prompts, task_prompts, features_xs,targets = collect_model_info_and_evaluate(model_checkpoint, exp_name = "UniSeg_Trainer", network = "3d_fullres", task = "Task097_11task", network_trainer = "UniSeg_Trainer", plans_identifier = "DoDNetPlans")


###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.UniSeg_Trainer.UniSeg_Trainer'>
For that I will be using the following configuration:
num_classes:  2
modalities:  {0: ['CT']}
use_mask_for_norm OrderedDict([(0, False), (1, False), (2, False), (3, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes None
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [4, 5, 5], 'patch_size': array([ 64, 192, 192]), 'median_patient_size_in_voxels': array([ 29, 113, 133]), 'current_spacing': array([3. , 1.5, 1.5]), 'original_spacing': array([3. , 1.5, 1.5]), 'do_dummy_2D_data_aug': False, 'pool_op_kernel_sizes': [[1, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]], 'conv_kernel_sizes': [[1, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]}

I am using stage 0 from these plans
I am using sample dice + CE loss


KeyboardInterrupt: 

In [ ]:
def plot_dist(s, nbins=30, c='b'):
    p, x = np.histogram(s, bins=nbins) # bin it into n = N//10 bins
    x = x[:-1] + (x[1] - x[0])/2   # convert bin edges to centers
    f = UnivariateSpline(x, p, s=nbins)
    plt.plot(x, f(x), c=c)

In [ ]:
target_lst = [t[-2] for t in targets]

tps = torch.vstack(task_prompts)
gts = torch.vstack(target_lst)
classifier = linear_probing(tps, gts)
predictions, accuracy = test_linear_classifier(classifier, tps, gts)

print(f"Linear Probing Accuracy: {accuracy}")

In [ ]:
pos_set, neg_set = extract(tps, gts)
pos_set = pos_set.detach().cpu().numpy().reshape(-1, 1)
neg_set = neg_set.detach().cpu().numpy().reshape(-1, 1)

In [ ]:
# plot_dist(np.concatenate((pos_set, neg_set)))
plt.close()
plot_dist(pos_set, c='b')
plot_dist(neg_set, c='r')
plt.show()
plt.savefig("/workspace/dist.png")

In [ ]:
# GMM
gmm, means, covariances = fit_gmm(pos_set, neg_set)

print(means)
print(covariances)

In [ ]:
# infer the assignment gmm made by visualization of distribution
pos_gt = np.zeros_like(pos_set)#.squeeze(1)
neg_gt = np.ones_like(neg_set)#.squeeze(1)

pos_gmm_acc = evaluate_gmm(gmm, pos_set, pos_gt)
neg_gmm_acc = evaluate_gmm(gmm, neg_set, neg_gt)
